In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
import gym

In [2]:
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

LTSM Autoencoder as feature detector
n Inputs must = n of Outputs
3 months - 90 days
input layer = 90 x 5 (low, high, open, close, vol)
hidden 1 = 32?
hidden 2 = 32?
output layer = 90 x 5

Building a recurrent autoencoder is straightforward: the encoder
is typically a sequence-to-vector RNN which compresses the input
sequence down to a single vector. The decoder is a vector-to-sequence RNN
that does the reverse:

In [6]:
n_features = 5 # (low, high, open, close, vol)
window_length = 90

recurrent_encoder = keras.models.Sequential([
keras.layers.LSTM(100, return_sequences=True, input_shape=[None, n_features]),
keras.layers.LSTM(30)
])

recurrent_decoder = keras.models.Sequential([
keras.layers.RepeatVector(n_features, input_shape=[30]),
keras.layers.LSTM(100, return_sequences=True),
keras.layers.TimeDistributed(keras.layers.Dense(n_features, activation="tanh"))
])
recurrent_ae = keras.models.Sequential([recurrent_encoder, recurrent_decoder])

In [8]:
recurrent_ae.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 30)                58120     
                                                                 
 sequential_2 (Sequential)   (None, 5, 5)              52905     
                                                                 
Total params: 111,025
Trainable params: 111,025
Non-trainable params: 0
_________________________________________________________________
